In [2]:
import json
import pandas as pd
import os

In [3]:
data_dir = '../data/raw/'
metadata_path = '../data/raw/data_list.csv'

file_path_list = []
for root, dirs, files in os.walk(data_dir):
    for file_name in files:
        if file_name.endswith(('hwp', 'pdf')):
            file_path_list.append(os.path.join(root, file_name))
        # if file_name.endswith(('csv','xlsx')):
print('텍스트 파일 수:', len(file_path_list))

텍스트 파일 수: 100


In [4]:
df = pd.read_csv(metadata_path)
print(len(df))
df.head(3)

100


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...


In [5]:
# 사업 요약을 기반으로 파일 추리고, 그중 query에 해당하는 청크 가져오기?
# !pip install olefile

In [8]:
import olefile
import zlib

def read_hwp_method1(file_path):
    """olefile로 HWP 읽기"""
    text_list = []

    with olefile.OleFileIO(file_path) as ole:
        # HWP 파일의 텍스트는 BodyText 섹션에 있음
        for entry in ole.listdir():
            if len(entry) > 1 and entry[0] == 'BodyText':
                try:
                    data = ole.openstream(entry).read()

                    # 압축 해제 시도
                    try:
                        unpacked = zlib.decompress(data, -15)
                        text = unpacked.decode('utf-16-le', errors='ignore')
                        # text = unpacked.decode('cp949', errors='ignore')
                    except:
                        text = data.decode('utf-16-le', errors='ignore')
                        # text = data.decode('cp949', errors='ignore')

                    text_list.append(text)
                except:
                    continue

    return ''.join(text_list)

In [9]:
test_file = "../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp"  # 실제 경로로 변경

print("=" * 80)
print("HWP 파일 읽기 테스트")
print("=" * 80)

# 방법 1 테스트 (olefile - 권장)
print("\n[방법 1] olefile + zlib")
try:
    text = read_hwp_method1(test_file)
    print(f"✅ 성공! 텍스트 길이: {len(text)}자")
    print(f"미리보기: {text[:10000]}")
except Exception as e:
    print(f"❌ 실패: {e}")

print("\n" + "=" * 80)


HWP 파일 읽기 테스트

[방법 1] olefile + zlib
✅ 성공! 텍스트 길이: 533748자
ф  ǜ хɀ  骘 ـ ـ Ր π   읐  чీ漠杳⌐Ъ䒎 䆠 䄯 ⛊ Â     镻䶍  K그림입니다. ـ ـ Ր π   읐  Bƀ      耀 ф  ǜ хɀ  邘 ـ ـ Ր π   읐  Bƀ	 ࠀ    耀 уĠ漠杳    ౅ɀ    d d U <   鎜  Bƀ      耀 ф  ǘ хɀ  團 ஸ ஸ ৶ ܈   읐  Bƀ      耀 ф  ǘ хɀ  様 ஸ ஸ ৶ ܈   읐  Bƀ      耀 уɀ2024. 03.桤灧    ︁贁贀ȀĀȀ̀ 䠀Ă     ĀĀ樀"됀︀︁贁贀ꨀ樀"     ࡂƀ耀      耀 ౄ  Ǚ ౅ɀ    d d U <   Ṭ  ࡈ˰    鞘 δ ǾǾ«鞘     䈀耈ā    Ā Ā  䐀而  䔀䀌   搀 搀 唀 㰀   鰀 ؀䠀̂   ĀȀĀȀºᘀ ︀︁贁贀ԀȀº     ࡂƀ    Î   耀 ృЀ2024년 ｢벤처확인종합관리시스템 기능 고도화｣ 용역사업
원본 그림의 이름: CLP00003684000d.bmp
ф  : хɀ    ฿ ฿ జ Ǆ Ĭ 쓸  чˠ氠瑢⌑ࠪ    옪 ഥ æ 묦埐   ࡍƀ؀  ࡈ˰  ԁ  옪 ഥ     옪     䈀耈ࠁ  ꄀȀ Ā  䌀态⸡ 鐀쓍᳉钬ෆ䐀 稀Ȁ 笀䔀䀌   똀᐀  ⠀Æ ؀䈀耀ā   ⤁Ā Ā   䐀耄  㨀 䔀䀄 ̀ﬀ쐀ⰀÄ ؀䈀耀ँ 쨀 Ā Ā  䌀 ଁ 扬t   ଀ഀ䐀耄  紀䔀䀄 ꌀ픀堀  ꀀ ؀䠀Ă   ࠁ ĀĀ␀退贀贀贀贀꼀␀     ၂ƀ耀      耀 ᑄ  ( ᑅɀ    Ϩ Ϩ ͒ ɘ   ֠  ࡈ˰  Ā 㡻 ݡ ­㡻     䈀耈ā  ∀Ā Ā  䐀而  ⠀ 䔀䀌   刀  怀7 ؀䠀Ă   ĀĀĀ笀8愀贀贀贀贀뀀笀8     ࡂƀ耀  Ģ   耀 ౄ  ( ౅ɀ    Ϩ Ϩ ͒ Ǵ   㝠  ࡈ˰  Ā

In [14]:
import olefile
import zlib
import struct
import re

def read_hwp_clean(file_path):
    """
    HWP 파일에서 깨끗한 텍스트만 추출

    Parameters:
    -----------
    file_path : str
        읽을 HWP 파일 경로

    Returns:
    --------
    str : 추출된 깨끗한 텍스트
    """
    text_list = []

    try:
        with olefile.OleFileIO(file_path) as ole:
            # BodyText/SectionX 스트림들을 찾아서 처리
            for entry in ole.listdir():
                if len(entry) > 1 and entry[0] == 'BodyText' and entry[1].startswith('Section'):
                    try:
                        data = ole.openstream(entry).read()

                        # 압축 해제
                        try:
                            unpacked = zlib.decompress(data, -15)
                        except:
                            unpacked = data

                        # HWP 텍스트 추출 (한글은 UTF-16LE)
                        text = extract_text_from_hwp_data(unpacked)
                        if text:
                            text_list.append(text)

                    except Exception as e:
                        continue

        full_text = '\n'.join(text_list)

        # 후처리: 불필요한 제어문자 제거
        full_text = clean_text(full_text)

        return full_text

    except Exception as e:
        print(f"HWP 파일 읽기 실패: {e}")
        return ""


def extract_text_from_hwp_data(data):
    """HWP 바이너리 데이터에서 텍스트 추출"""
    text_parts = []
    pos = 0

    while pos < len(data):
        try:
            # HWP 레코드 헤더 읽기 (태그 정보)
            if pos + 4 > len(data):
                break

            # 4바이트 헤더
            header = struct.unpack('<I', data[pos:pos+4])[0]
            tag_id = header & 0x3FF
            level = (header >> 10) & 0x3FF
            size = (header >> 20) & 0xFFF

            pos += 4

            # HWPTAG_PARA_TEXT (0x50) - 문단 텍스트
            if tag_id == 0x50:
                if pos + size <= len(data):
                    try:
                        # UTF-16LE로 디코딩
                        text = data[pos:pos+size].decode('utf-16le', errors='ignore')
                        # 실제 텍스트만 필터링
                        text = ''.join(c for c in text if c.isprintable() or c in '\n\r\t ')
                        if text.strip():
                            text_parts.append(text)
                    except:
                        pass

            pos += size

        except:
            pos += 1

    return '\n'.join(text_parts)


def clean_text(text):
    """텍스트 후처리 - 불필요한 문자 제거"""
    # NULL 문자 제거
    text = text.replace('\x00', '')

    # 연속된 공백을 하나로
    text = re.sub(r' +', ' ', text)

    # 연속된 줄바꿈을 최대 2개로
    text = re.sub(r'\n{3,}', '\n\n', text)

    # 앞뒤 공백 제거
    text = text.strip()

    return text


# ============================================================================
# 대안: 더 간단하지만 덜 정확한 방법
# ============================================================================

def read_hwp_simple(file_path):
    """
    간단한 방법 - 모든 출력 가능한 문자만 추출
    정확도는 낮지만 빠르고 안정적
    """
    text_list = []

    try:
        with olefile.OleFileIO(file_path) as ole:
            for entry in ole.listdir():
                if len(entry) > 1 and entry[0] == 'BodyText':
                    try:
                        data = ole.openstream(entry).read()

                        # 압축 해제 시도
                        try:
                            unpacked = zlib.decompress(data, -15)
                        except:
                            unpacked = data

                        # UTF-16LE 디코딩 시도
                        try:
                            text = unpacked.decode('utf-16le', errors='ignore')
                        except:
                            text = unpacked.decode('utf-8', errors='ignore')

                        # 출력 가능한 문자만 추출
                        clean = ''.join(c for c in text if c.isprintable() or c in '\n\r\t ')

                        # 한글만 포함된 부분 찾기
                        korean_parts = re.findall(r'[가-힣a-zA-Z0-9\s\.,\(\)]+', clean)
                        text_list.extend([p for p in korean_parts if len(p) > 10])

                    except:
                        continue

        return '\n'.join(text_list)

    except Exception as e:
        print(f"HWP 파일 읽기 실패: {e}")
        return ""


# ============================================================================
# 테스트 코드
# ============================================================================

if __name__ == "__main__":
    import os

    # 테스트할 파일 경로
    test_file = "../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp"

    if os.path.exists(test_file):
        print("=" * 80)
        print("[방법 1] 정교한 추출 (read_hwp_clean)")
        print("=" * 80)
        text1 = read_hwp_clean(test_file)
        print(f"텍스트 길이: {len(text1)}자")
        print(f"\n미리보기 (처음 500자):\n{text1[:500]}\n")

        print("=" * 80)
        print("[방법 2] 간단한 추출 (read_hwp_simple)")
        print("=" * 80)
        text2 = read_hwp_simple(test_file)
        print(f"텍스트 길이: {len(text2)}자")
        print(f"\n미리보기 (처음 500자):\n{text2}\n")
    else:
        print(f"파일을 찾을 수 없습니다: {test_file}")
        print("\n실제 파일 경로를 입력하고 다시 실행하세요.")


# ============================================================================
# 사용 예제
# ============================================================================

"""
# 단일 파일 읽기
text = read_hwp_clean('제안서.hwp')
print(text)

# 또는 간단한 방법
text = read_hwp_simple('제안서.hwp')
print(text)

# 여러 파일 처리
import pandas as pd

results = []
for file in hwp_files:
    text = read_hwp_simple(file)
    results.append({
        'filename': file,
        'text': text,
        'length': len(text)
    })

df = pd.DataFrame(results)
"""

[방법 1] 정교한 추출 (read_hwp_clean)
텍스트 길이: 0자

미리보기 (처음 500자):


[방법 2] 간단한 추출 (read_hwp_simple)
텍스트 길이: 41960자

미리보기 (처음 500자):
벤처확인종합관리시스템 기능 고도화
(복수의결권주식, 스톡옵션, 성과조건부주식) 
K그림입니다.
원본 그림의 이름
 CLP00003684000d.bmp
원본 그림의 크기
 가로 691pixel, 세로 411pixel
 4. 제안요청내용	
 5. 입찰관련사항	
 6. 제안서작성요령	
 7. 별지서식 및 붙임	
벤처기업육성에 관한 특별조치법
) 복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관, 신규 구축업무를 
) 벤처기업법 제16조의11에 따라 발행된 복수의결권주식 보고 업무처리 시스템 구축
 모든 주주는 1주당 하나의 의결권을 갖는 주주평등원칙(상법)과 별도로 비상장 벤처기업 창업자에게만 1주당 최대 10배의 의결권 행사를 부여하는 제도 
) 벤처기업법 제16조의3에 따라 부여된 벤처기업 스톡옵션 부여, 취소
철회 신고 및 업무시스템 구축
 비상장 벤처기업 임
직원 및 기업 성장에 기여한 자에게 미리 정한 가격(행사가격)으로 신주를 인수하거나 자기의 주식을 매수할 수 있는 권리 혹은 주식의 시가와 행사가격의 차액을 청구할 수 있는 권리를 부여하는 제도
 스톡옵션 신청 기능 이관을 추진하며, 과거 데이터 정제 및 적재 등을 통한 데이터 이관 추진
) 벤처기업법 제16조의19에 따라 성과조건부주식교부계약의 신고 및 업무처리 시스템 구축
 비상장 벤처기업 임
직원 및 특정인에게 자기주식 부여를 약속하되, 일정 재직기간 및 조건을 충족해야 주식이 부여 대상자에게 귀속되도록 제한을 둔 주식
 복수의결권주식 발행 보고 및 스톡옵션 신고기능은 기존 시스템 내 벤처확인 이력정보와 신고자(벤처기업)의 정보와 연동
 (대기업 참여 제한) 
제48조제2항에 따라, 사업금액 20억원 미만 계약은 대기업 참여 제한


"\n# 단일 파일 읽기\ntext = read_hwp_clean('제안서.hwp')\nprint(text)\n\n# 또는 간단한 방법\ntext = read_hwp_simple('제안서.hwp')\nprint(text)\n\n# 여러 파일 처리\nimport pandas as pd\n\nresults = []\nfor file in hwp_files:\n    text = read_hwp_simple(file)\n    results.append({\n        'filename': file,\n        'text': text,\n        'length': len(text)\n    })\n\ndf = pd.DataFrame(results)\n"

In [ ]:
import pyhwpx

In [13]:
from pyhwpx import Hwp
import os
import time
import pythoncom
from pywintypes import com_error
# from pyhwpx.const import MsoEncodingEUCKR

hwpx= Hwp()
file_path =  "../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp"

try:
    hwpx.Open(file_path)
    print(f"✅ HWP 파일을 성공적으로 열었습니다: {file_path}")

    time.sleep(10)
    print("파일 로딩 시간 확보 완료.")
    # 4. 문서 전체 선택 및 복사
    # hwpx.Run("AllSelect")
    # hwpx.Run("Copy")
    # print("✅ 문서 전체 선택 및 복사 액션 성공")

    # full_text = hwpx.get_text_file("Text")
    # full_text = hwpx.get_text_file("UNICODE")
    full_text = hwpx.get_text_file()
    # full_text = hwpx.GetTextFile("UNICODE")
    print("\n--- 추출된 텍스트 일부 ---")
    print(full_text)
    hwpx.Quit()

except com_error as e:
    print(f"❌ HWP 액션 실행 중 오류 발생. 한글 창을 모두 닫고 다시 시도하거나, 파일 경로를 확인하세요. 오류 코드: {e}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}. 파일 이름을 확인하세요.")

✅ HWP 파일을 성공적으로 열었습니다: ../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
파일 로딩 시간 확보 완료.

--- 추출된 텍스트 일부 ---
None
❌ HWP 액션 실행 중 오류 발생. 한글 창을 모두 닫고 다시 시도하거나, 파일 경로를 확인하세요. 오류 코드: (-2147023179, '알 수 없는 인터페이스입니다.', None, None)


In [1]:
from pyhwpx import Hwp
import os
import time
import pythoncom
from pywintypes import com_error
# from pyhwpx.const import MsoEncodingEUCKR

hwpx= Hwp()
file_path =  "../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp"
hwpx.open(file_path)
print(f"✅ HWP 파일을 성공적으로 열었습니다: {file_path}")

time.sleep(2)
extracted_text = hwpx.get_text_file()
print(extracted_text)

✅ HWP 파일을 성공적으로 열었습니다: ../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
None


In [1]:
from pyhwpx import Hwp
import os
import time
import pythoncom
from pywintypes import com_error

# 1. COM 초기화
try:
    pythoncom.CoInitialize()
except com_error:
    # 이미 초기화되어 있을 경우 무시
    pass

hwpx = Hwp()
file_path =  "../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp"
output_file_path = "extracted_text.txt"

try:
    # 2. 파일 열기
    hwpx.open(file_path)
    print(f"✅ HWP 파일을 성공적으로 열었습니다: {file_path}")

    # 3. 충분한 시간 지연 (예: 3초)
    time.sleep(3)

    # 4. 텍스트 추출 시도
    extracted_text = hwpx.get_text_file()

    if extracted_text is not None:
        # 5. 추출된 텍스트 저장
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(extracted_text)
        print(f"✅ HWP 문서의 텍스트를 성공적으로 추출하여 저장했습니다: {output_file_path}")
    else:
        # 텍스트 추출이 실패했을 경우 (None 반환)
        print("❌ 경고: 텍스트 추출에 실패했습니다. (get_text_file()에서 None 반환)")
        print("     - 파일 로드 시간이 부족하거나, 한글 프로그램에 팝업 창이 떴을 수 있습니다.")

except com_error as e:
    print(f"❌ COM 오류 발생: {e}")
except Exception as e:
    print(f"❌ 예외 발생: {e}")

finally:
    # 6. Hwp 객체 종료
    if 'hwpx' in locals() and hwpx.IsOpened():
        hwpx.quit()
        print("✅ Hwp 객체를 종료했습니다.")

✅ HWP 파일을 성공적으로 열었습니다: ../data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
❌ 경고: 텍스트 추출에 실패했습니다. (get_text_file()에서 None 반환)
     - 파일 로드 시간이 부족하거나, 한글 프로그램에 팝업 창이 떴을 수 있습니다.


AttributeError: 'Hwp' object has no attribute 'IsOpened'

In [ ]:
from pyhwpx import Hwp
import os
import time
import pythoncom
from pywintypes import com_error

def convert_hwp_to_pdf(hwp_path, pdf_path):
    """지정된 HWP 파일을 PDF 파일로 변환합니다."""

    # 1. COM 초기화 (pyhwpx 사용 시 필수)
    try:
        pythoncom.CoInitialize()
    except com_error:
        pass # 이미 초기화된 경우 무시

    hwpx = Hwp()

    try:
        # 2. 메시지 박스 모드 설정 (경고/팝업 자동 닫기)
        # HWP 파일 열기 시 발생하는 각종 팝업으로 인해 스크립트가 멈추는 것을 방지
        hwpx.SetMessageBoxMode(0)

        # 3. HWP 파일 열기
        success = hwpx.open(hwp_path)

        if success:
            print(f"✅ HWP 파일 열기 성공: {hwp_path}")

            # 4. 파일 로드 및 COM 연결 안정화를 위한 지연 시간 (3초)
            # 10초를 줬음에도 안 되었다면, 이 시간보다는 SetMessageBoxMode(0)이 핵심입니다.
            time.sleep(3)

            # 5. PDF로 저장 (핵심 기능)
            # SaveAs(저장할 경로, 저장 형식, 옵션)
            # 저장 형식 "PDF"는 한글 컨트롤에서 PDF로 변환하도록 지시합니다.
            hwpx.SaveAs(pdf_path, "PDF", "")

            print(f"✅ PDF 변환 성공 및 저장: {pdf_path}")
            return True
        else:
            print(f"❌ 파일 열기 실패. 경로를 확인하거나 파일에 오류가 있는지 확인하세요: {hwp_path}")
            return False

    except com_error as e:
        print(f"❌ COM 오류 발생 (한글 프로그램 연결 실패 또는 오류): {e}")
        return False
    except Exception as e:
        print(f"❌ 예외 발생: {e}")
        return False

    finally:
        # 6. Hwp 객체 종료
        if 'hwpx' in locals() and hwpx.IsOpened():
            hwpx.quit()
            print("✅ Hwp 객체를 종료했습니다.")
        # 모든 작업 후 COM 해제는 필수가 아니지만 안전을 위해 추가 가능
        # pythoncom.CoUninitialize()